# Module 3 — Program A: CNN Building Blocks (Convolution • Filters • Padding • Stride • Pooling)

**Aim:** Understand and experiment with the core building blocks of CNNs through hands‑on demos and a small MNIST classifier.  
**Covers:** 2D convolution, custom filters (edge/blur), padding (`valid` vs `same`), stride effect on output size, max vs avg pooling, feature maps.

> Dataset: **MNIST** (loaded from Keras; no internet needed)


In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Load one sample image for filter demos
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
img = x_train[0].astype('float32')/255.0
img = np.expand_dims(img, axis=(0, -1))  # shape (1, 28, 28, 1)
print('Demo image shape:', img.shape)

## 1) Filters and Convolution via `tf.nn.conv2d`

In [ ]:
# Define common 3x3 filters: edge detect (Sobel-like) and blur
edge_filter = np.array([[ -1, -1, -1],
                        [ -1,  8, -1],
                        [ -1, -1, -1 ]], dtype='float32')
blur_filter = (1/9) * np.ones((3,3), dtype='float32')

# Prepare filters for conv2d: [kh, kw, in_ch, out_ch]
edge_kernel = edge_filter.reshape(3,3,1,1)
blur_kernel = blur_filter.reshape(3,3,1,1)

# Convolutions with SAME padding and stride 1
edge_out = tf.nn.conv2d(img, edge_kernel, strides=1, padding='SAME')
blur_out = tf.nn.conv2d(img, blur_kernel, strides=1, padding='SAME')

fig, axs = plt.subplots(1,3, figsize=(9,3))
axs[0].imshow(img[0,...,0], cmap='gray'); axs[0].set_title('Original'); axs[0].axis('off')
axs[1].imshow(edge_out.numpy()[0,...,0], cmap='gray'); axs[1].set_title('Edge conv'); axs[1].axis('off')
axs[2].imshow(blur_out.numpy()[0,...,0], cmap='gray'); axs[2].set_title('Blur conv'); axs[2].axis('off')
plt.show()

## 2) Padding & Stride: effect on output size

In [ ]:
def conv_shape(h, w, k=3, s=1, padding='valid'):
    if padding.lower()=='valid':
        out_h = (h - k)//s + 1
        out_w = (w - k)//s + 1
    else:  # same
        out_h = int(np.ceil(h / s))
        out_w = int(np.ceil(w / s))
    return out_h, out_w

for padding in ['valid','same']:
    for s in [1,2]:
        oh, ow = conv_shape(28, 28, k=3, s=s, padding=padding)
        print(f'padding={padding:5s} stride={s} -> out=({oh},{ow})')

## 3) Pooling: Max vs Average

In [ ]:
max_pooled = tf.nn.max_pool2d(img, ksize=2, strides=2, padding='VALID')
avg_pooled = tf.nn.avg_pool2d(img, ksize=2, strides=2, padding='VALID')

fig, axs = plt.subplots(1,3, figsize=(9,3))
axs[0].imshow(img[0,...,0], cmap='gray'); axs[0].set_title('Original'); axs[0].axis('off')
axs[1].imshow(max_pooled.numpy()[0,...,0], cmap='gray'); axs[1].set_title('MaxPool 2x2'); axs[1].axis('off')
axs[2].imshow(avg_pooled.numpy()[0,...,0], cmap='gray'); axs[2].set_title('AvgPool 2x2'); axs[2].axis('off')
plt.show()

## 4) Mini Classifier on MNIST with Config Variants

In [ ]:
# Prepare dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = (x_train.astype('float32')/255.0)[..., None]
x_test  = (x_test.astype('float32')/255.0)[..., None]

def build_cnn(padding='same', stride=1, pooling='max'):
    inputs = tf.keras.Input(shape=(28,28,1))
    x = tf.keras.layers.Conv2D(16, 3, strides=stride, padding=padding, activation='relu')(inputs)
    x = tf.keras.layers.Conv2D(32, 3, strides=1, padding=padding, activation='relu')(x)
    if pooling=='max':
        x = tf.keras.layers.MaxPool2D()(x)
    else:
        x = tf.keras.layers.AveragePooling2D()(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    outputs = tf.keras.layers.Dense(10, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

configs = [
    ('same', 1, 'max'),
    ('same', 2, 'max'),
    ('valid',1, 'max'),
    ('same', 1, 'avg')
]

hist = {}; test_acc = {}
for padding, stride, pool in configs:
    key = f'pad={padding}|stride={stride}|pool={pool}'
    m = build_cnn(padding, stride, pool)
    h = m.fit(x_train, y_train, validation_split=0.1, epochs=3, batch_size=128, verbose=1)
    hist[key] = h
    test_acc[key] = m.evaluate(x_test, y_test, verbose=0)[1]
print('Test accuracy by config:', {k: round(v,4) for k,v in test_acc.items()})

In [ ]:
# Plot validation accuracy for all configs
fig, ax = plt.subplots()
for name, h in hist.items():
    ax.plot(h.history['val_accuracy'], label=name)
ax.set_xlabel('Epoch'); ax.set_ylabel('Val Accuracy'); ax.set_title('Config Comparison')
ax.legend(); plt.show()

### Result & Inference (to be written)
- Discuss how **padding/stride** change output resolution and accuracy.
- Compare **max vs avg pooling** impacts.
- Attach 1–2 feature map screenshots and infer what early filters learn.
